In [1]:
from stratopy import metadatatools
from stratopy.extractors import cloudsat
from stratopy.extractors import goes
from stratopy.transformers import mergers
from stratopy.transformers import scalers

Download one band file from ABI GOES16

We will download band 13 of the L2 product "CMIPF" (Cloud and Moisture Full Disk) from AWS server.
For that, we have to create a goes_obj, indicating the product type "ABI-L2-CMIPF" and the channel (13) in order to stablish a connection.  

In [2]:
goes_obj = goes.GOES16(product_type="ABI-L2-CMIPF", channel=13)
goes_obj 

<GOES16 product_type='ABI-L2-CMIPF', ch=13>

Then, we choose a date and time for downloading, and we fetch the product for the server. If the specified date-time is not available, it will fetch the product of the nearest date-time available.

In [3]:
goes_data = goes_obj.fetch("2019 jan 2 19:30")
goes_data

<SatelliteData products_keys=('CMI',), instruments_types=('Radiometer',), platforms=('GOES',), orbits_types=('Geostationary',), times_starts=('2019-01-02T19:00:36.3Z',), times_ends=('2019-01-02T19:11:14.1Z',)>

To access the Xarray original data, we use the method .data  

In [4]:
goes_data.data

<xarray.Dataset>
Dimensions:                                 (y: 5424, x: 5424,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                       datetime64[ns] ...
  * y                                       (y) float64 0.1518 ... -0.1518
  * x                                       (x) float64 -0.1518 ... 0.1518
    y_image                                 float32 ...
    x_image                                 float32 ...
    band_wavelength                         (band) float32 ...
    band_id                                 (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/29)
    CMI                                     (y, x) float32 ...
    DQF                                     (y, x) float32 ...
    time_bounds                             (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                  int32 ...
    y_image_bounds                          (number_of_image_bounds) float32 ...
    x_image_bounds                          (number_of_image_bounds) float32 ...
    ...                                      ...
    algorithm_dynamic_input_data_container  int32 ...
    percent_uncorrectable_GRB_errors        float32 ...
    percent_uncorrectable_L0_errors         float32 ...
    earth_sun_distance_anomaly_in_AU        float32 ...
    processing_parm_version_container       int32 ...
    algorithm_product_version_container     int32 ...
Attributes: (12/29)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v25, 05 July 2013)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    cdm_data_type:             Image
    time_coverage_start:       2019-01-02T19:00:36.3Z
    time_coverage_end:         2019-01-02T19:11:14.1Z
    timeline_id:               ABI Mode 3
    production_data_source:    Realtime
    id:                        5c74601f-92ef-4f97-ae10-919b95261557

Lets see one transformer. This one normalizes an ABI image to [0,1].

Returns an xarray with an image of shape (n_bands, image_width, image_height).

In [5]:
norm_goes = scalers.MinMaxNormalize().transform(goes_data)
norm_goes

<SatelliteData products_keys=('CMI',), instruments_types=('Radiometer',), platforms=('GOES',), orbits_types=('Geostationary',), times_starts=('2019-01-02T19:00:36.3Z',), times_ends=('2019-01-02T19:11:14.1Z',)>

In [6]:
norm_goes.data.CMI.data[:,2500,2500]

array([0.7556311], dtype=float32)

Download a file from CloudSat.

In [7]:
#from unittest import mock

#with mock.patch("paramiko.RSAKey.from_private_key_file", return_value="pkey"):
#    with mock.patch("paramiko.SSHClient.connect"):
#        cs_obj = cloudsat.CloudSat(
#        product_type="2B-CLDCLASS.P1_R05",
#        username="fakeusr@gmail.edu",
#        keyfile="some_file",
#        )
        
#        csat_data = cs_obj.fetch("2019 dec 12 15:00")

In [8]:
CLOUDSAT_PATH = "../../../tests/data/CloudSat/2019002175851_67551_CS_2B-CLDCLASS_GRANULE_P1_R05_E08_F03.hdf"
csat_data = cloudsat.read_hdf4(CLOUDSAT_PATH)

In [9]:
csat_sat_data = cloudsat.read_as_SatelliteData(CLOUDSAT_PATH, "2B-CLDCLASS.P1_R05")
csat_sat_data

<SatelliteData products_keys=('2B-CLDCLASS.P1_R05',), instruments_types=('Radar',), platforms=('CloudSat',), orbits_types=('Polar',), times_starts=('2019-01-02T17:59:01.740000+00:00',), times_ends=('2019-01-02T19:37:33.579844+00:00',)>

Merge products

In [10]:
merged_data = mergers.MergePolarGeos("2019 2 jan 18:30", 
                                     time_zone="UTC", 
                                     trim_size=(3,3)).transform(csat_sat_data, norm_goes)

In [11]:
merged_data

<xarray.Dataset>
Dimensions:           (cloudsat_trace: 36950, z: 125, layer: 10, nbands: 1,
                       img_wide: 3, img_height: 3)
Coordinates:
    trace             object <built-in method copy of numpy.ndarray object at...
    height            (cloudsat_trace, z) int16 -9999 -9999 ... -9999 -9999
    layers            object <built-in method copy of numpy.ndarray object at...
    lat               (cloudsat_trace) float64 -0.009365 -0.01907 ... 0.004176
    lon               (cloudsat_trace) float64 117.1 117.1 117.1 ... 92.5 92.5
    time              (cloudsat_trace) datetime64[ns] 2019-01-02T17:59:01.740...
    precip_flag       (cloudsat_trace) int8 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9
    land_sea_flag     (cloudsat_trace) float32 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0
  * cloudsat_trace    (cloudsat_trace) int32 0 1 2 3 ... 36946 36947 36948 36949
  * nbands            (nbands) int8 1
  * img_wide          (img_wide) int8 1 2 3
  * img_height        (img_height) int8 1 2 3
Dimensions without coordinates: z, layer
Data variables:
    cloud_scenario    (cloudsat_trace, z) int16 2081 2081 2081 ... 2113 2113
    cloud_layer_type  (cloudsat_trace, layer) int8 0 0 0 0 0 0 0 ... 0 0 0 0 0 0
    cloud_layer_base  (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    cloud_layer_top   (cloudsat_trace, layer) float32 -99.0 -99.0 ... -99.0
    geos              (cloudsat_trace, nbands, img_wide, img_height) float32 ...
Attributes:
    TAIstart:             820605541.74
    UTCstart:             [[64731.73828125]]
    bin_size:             [239.82907104]
    time_coverage_start:  2019-01-02T17:59:01.740000+00:00
    time_coverage_end:    2019-01-02T19:37:33.579844+00:00
    platform_ID:          CloudSat